# Reccomendation system

In [ ]:
pip install scikit-surprise

     |████████████████████████████████| 11.8MB 9.7MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1617588 sha256=6065251c3755ff4c8a5eaf590cab4ac0fecdc26592c261867b79cc37ec364686
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


## Packages

In [ ]:
import pandas as pd
from surprise import Dataset
from surprise import Reader
from surprise import NMF, SVD, KNNWithMeans
from surprise.model_selection import cross_validate
from surprise import Reader, Dataset
from surprise import KNNWithMeans
from surprise import Dataset
from surprise.model_selection import GridSearchCV
from surprise import KNNWithMeans
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import GridSearchCV
from surprise import KNNWithMeans
from surprise.model_selection import train_test_split
from surprise import accuracy

## Data

In [ ]:
# This is the same data that was plotted for similarity earlier
# with one new user "E" who has rated only movie 1
ratings_dict = {
    "user_id": ['A', 'A', 'B', 'B', 'C', 'C', 'D', 'D', 'E'],
    "item_id": [1, 2, 1, 2, 1, 2, 1, 2, 1],
    "rating": [1, 2, 2, 4, 2.5, 4, 4.5, 5, 3],
}
df = pd.DataFrame(ratings_dict)
reader = Reader(rating_scale=(1, 5))



# Loads Pandas dataframe
#data = Dataset.load_from_df(df[['user_id','item_id','rating']], reader)

# Loads the builtin Movielens-100k data
data = Dataset.load_builtin('ml-100k')
trainset, testset = train_test_split(data, test_size=.25)


dataframe1=pd.DataFrame(data.__dict__['raw_ratings'], columns=['user_id','item_id','rating','timestamp'])
dataframe1[['user_id','item_id','rating']]

,user_id,item_id,rating
0,196,242,3.0
1,186,302,3.0
2,22,377,1.0
3,244,51,2.0
4,166,346,1.0
...,...,...,...
99995,880,476,3.0
99996,716,204,5.0
99997,276,1090,1.0
99998,13,225,2.0


## Model selection


In [ ]:
cv = []
# Iterate over all recommender system algorithms
for recsys in [NMF(), SVD(), KNNWithMeans()]:
    # Perform cross validation
    tmp = cross_validate(recsys, data, measures=['RMSE'], cv=5, verbose=False)
    cv.append((str(recsys).split(' ')[0].split('.')[-1], tmp['test_rmse'].mean()))
pd.DataFrame(cv, columns=['RecSys', 'RMSE'])

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.


,RecSys,RMSE
0,NMF,0.964554
1,SVD,0.936290
2,KNNWithMeans,0.951956


## KNN Means

In [ ]:
sim_options = {
    "name": ["msd", "cosine"],
    "min_support": [3, 4, 5],
    "user_based": [False, True],
}

param_grid = {"sim_options": sim_options}
gs1 = GridSearchCV(KNNWithMeans, param_grid, measures=["rmse", "mae"], cv=3)
gs1.fit(data)


print(gs1.best_score["rmse"])
print(gs1.best_params["rmse"])

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computi

In [ ]:
# To use item-based cosine similarity
sim_options = {
    "name": "msd",
    "min_support": 3,
    "user_based": True,  # Compute  similarities between items
}

algo1 = KNNWithMeans(sim_options=sim_options)
trainingSet = trainset
algo1.fit(trainingSet)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [ ]:
predictions1 = algo1.test(testset)
accuracy.rmse(predictions1)

RMSE: 0.9613


0.9612825502037637

In [ ]:
prediction11 = algo1.predict('716','204')
prediction11.est

4.224530417701448

In [ ]:
titles = pd.DataFrame(dataframe1['item_id'].unique(),columns=['item_id'])

titles['Estimate_Score'] = titles['item_id'].apply(lambda x: algo1.predict('716', x).est)
titles = titles.sort_values(by=['Estimate_Score'], ascending=False)
a=titles.head(20)
print(a)

     item_id  Estimate_Score
1560    1233        5.000000
1239    1367        5.000000
1579    1653        5.000000
1571    1642        5.000000
1566    1656        5.000000
1135     119        5.000000
1150     814        5.000000
1436    1293        5.000000
1426    1080        5.000000
1272     247        5.000000
1297    1536        5.000000
1323    1398        5.000000
1333    1554        5.000000
1619    1467        5.000000
1641    1631        4.996183
1666    1636        4.996183
1552    1651        4.996183
1674    1645        4.996183
1591    1650        4.996183
1038    1189        4.960239


## SVD

In [ ]:
param_grid = {
    "lr_bu": [0.002, 0.005],
    "lr_bi": [0.002, 0.005],
    "lr_pu": [0.002, 0.005],
    "lr_qi": [0.002, 0.005],
}
gs2 = GridSearchCV(SVD, param_grid, measures=["rmse", "mae"], cv=3)
gs2.fit(data)


print(gs2.best_score["rmse"])
print(gs2.best_params["rmse"])

1.1834517962186384
{'lr_bu': 0.005, 'lr_bi': 0.005, 'lr_pu': 0.005, 'lr_qi': 0.005}


In [ ]:
algo2 = SVD(n_epochs=400, lr_bu=0.005,lr_bi=0.005,lr_pu=0.005,lr_qi=0.005)
trainingSet = trainset
algo2.fit(trainingSet)

In [ ]:
predictions2 = algo2.test(testset)
accuracy.rmse(predictions2)

RMSE: 0.9768


0.976845885695805

In [ ]:
prediction2 = algo2.predict('716','204')
prediction2.est

4.78233518527119

In [ ]:
titles = pd.DataFrame(dataframe1['item_id'].unique(),columns=['item_id'])

titles['Estimate_Score'] = titles['item_id'].apply(lambda x: algo2.predict('716', x).est) 
titles = titles.sort_values(by=['Estimate_Score'], ascending=False)
b=titles.head(20)
print(b)

    item_id  Estimate_Score
936     136        5.000000
31       98        5.000000
240     357        5.000000
200     318        5.000000
239      64        5.000000
68       15        5.000000
185     178        5.000000
538     511        5.000000
101     174        5.000000
277     169        5.000000
9        86        5.000000
66      427        5.000000
89      275        4.980100
62      526        4.973715
78      480        4.970231
21      486        4.968877
485     487        4.955554
166     216        4.954819
99      496        4.942952
666     898        4.942533


## NMF

In [ ]:
param_grid = {
    "lr_bu": [0.002, 0.005],
    "lr_bi": [0.002, 0.005],
}
gs3 = GridSearchCV(NMF, param_grid, measures=["rmse", "mae"], cv=3)
gs3.fit(data)


print(gs3.best_score["rmse"])
print(gs3.best_params["rmse"])

1.3172200188118472
{'lr_bu': 0.002, 'lr_bi': 0.002}


In [ ]:
algo3 = NMF(n_epochs=200, lr_bu=0.002,lr_bi=0.002)
trainingSet = trainset
algo3.fit(trainingSet)

In [ ]:
predictions3 = algo3.test(testset)
accuracy.rmse(predictions3)

RMSE: 0.9638


0.963755043719562

In [ ]:
prediction3 = algo3.predict('716','204')
prediction3.est

3.9535203234392133

In [ ]:
titles = pd.DataFrame(dataframe1['item_id'].unique(),columns=['item_id'])

titles['Estimate_Score'] = titles['item_id'].apply(lambda x: algo3.predict('716', x).est)
titles = titles.sort_values(by=['Estimate_Score'], ascending=False)
c=titles.head(20)
print(c)

     item_id  Estimate_Score
843     1194        4.954226
200      318        4.943947
170       48        4.933218
524      543        4.884201
1608    1643        4.882286
991     1131        4.881454
1323    1398        4.880845
239       64        4.863550
99       496        4.844015
66       427        4.826014
180      408        4.815860
1571    1642        4.782504
719      313        4.755587
189      127        4.749261
660      657        4.746541
56       143        4.740092
1559    1251        4.734754
1078    1062        4.700144
1591    1650        4.696259
1666    1636        4.695663


## Results

In [ ]:
print(a)

     item_id  Estimate_Score
1297    1536        5.000000
1646    1122        5.000000
1647    1201        5.000000
1150     814        5.000000
1619    1467        5.000000
1255    1449        5.000000
1579    1653        5.000000
1038    1189        5.000000
1130    1500        5.000000
1287    1125        5.000000
1562    1599        5.000000
1414    1537        4.910921
512      285        4.788870
673      641        4.779476
125      479        4.736742
1131    1463        4.676369
1428    1512        4.676040
436      661        4.673216
574       59        4.612717
359      187        4.596261


In [ ]:
print(b)

     item_id  Estimate_Score
1196     236        5.000000
438      692        4.935115
550      663        4.905844
512      285        4.903552
834       67        4.893054
389        8        4.854367
36       603        4.849368
221      153        4.828581
289      286        4.827380
431      655        4.822139
475      506        4.781670
57       423        4.773410
482      190        4.771102
260      134        4.734874
249       33        4.723849
496      654        4.722235
100      132        4.702856
574       59        4.676888
612      114        4.668941
216       56        4.650536


In [ ]:
print(c)

     item_id  Estimate_Score
1560    1233        5.000000
996     1242        5.000000
1001     954        5.000000
1098     902        5.000000
569      963        5.000000
1084     936        5.000000
1473     899        4.934411
320       97        4.871460
66       427        4.833961
71       165        4.828879
1051    1172        4.768076
225      507        4.765972
61       219        4.761520
240      357        4.728081
1619    1467        4.719639
1357    1103        4.711613
200      318        4.693205
1325     534        4.690482
31        98        4.633668
160      673        4.628519


In [ ]:
print(dataframe1.loc[2600,['user_id','item_id','rating']])

user_id     34
item_id    242
rating       5
Name: 2600, dtype: object


In [ ]:
user="34"
item="242"
prediction1 = algo1.predict(user,item)
prediction2 = algo2.predict(user,item)
prediction3 = algo3.predict(user,item)
print(prediction1.est)
print(prediction2.est)
print(prediction3.est)

4.651991838853594
4.927318400649833
4.647846451535687
